In [1]:
import pandas as pd
import numpy as np

In [154]:
df = pd.read_csv('/content/t_bg_wf.csv').set_index('id')
np.random.seed(10)
alpha = np.random.dirichlet(np.ones(11), size=1, ).squeeze()
groups = {f'omega_{k_id+1}': [] for k_id in range(11)}
k = 1000
acc_columns = df.columns.tolist()[:-1]
n_select = (alpha * k).astype('int')

In [155]:
count = 0
for _ in range(df.shape[0]):
    ks = df.sum(axis=0).values/alpha
    ks_idx = ks.argsort()
    df['column_sum'] = df.sum(axis=1)
    df.sort_values(by='column_sum', inplace=True)
    if count != df.shape[1]-1 and df.sum().sum() > 0:
        if (df.loc[:, f'wf_{ks_idx[count]+1}'] == 1).sum() == 0: count += 1
        drop_idx = df.loc[df.loc[:, f'wf_{ks_idx[count]+1}']==1, :].index[0]
        if len(groups[f'omega_{ks_idx[count]+1}']) < n_select[ks_idx[count]]:
            groups[f'omega_{ks_idx[count]+1}'].append((drop_idx, df.loc[drop_idx, acc_columns].values))
        df.drop(drop_idx, inplace=True, axis=0)
    df.drop('column_sum', inplace=True, axis=1)

In [156]:
(alpha * k).astype('int')

array([186,   2, 126, 174,  87,  32,  27, 180,  23,  11, 146])

In [157]:
for k, v in groups.items(): print(k, len(v))

omega_1 151
omega_2 2
omega_3 93
omega_4 174
omega_5 58
omega_6 30
omega_7 27
omega_8 180
omega_9 23
omega_10 11
omega_11 88


In [158]:
final_matrix = np.zeros((1000, 11))
for i, (k, v) in enumerate(groups.items()):
    final_matrix[list(map(lambda x: x[0] - 1, v)), i] = 1

In [160]:
final_matrix.sum(axis=0)

array([151.,   2.,  93., 174.,  58.,  30.,  27., 180.,  23.,  11.,  88.])

In [162]:
np.savetxt('output_file.txt', final_matrix)